In [1]:
import pandas as pd
import re
import torch

from emotion_detection.loader import create_data_loader, split_data
from emotion_detection.models import TextClassificationParsBert, pytorch_model

from transformers import BertConfig, BertTokenizer, BertModel

## hyper-parameters

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

max_len = 128
train_batch_size = 128
valid_batch_size = 128
test_batch_size = 128

epoch = 3
EEVERY_EPOCH = 1000
lr = 2e-5
CLIP = 0.0


##  dataset process

In [3]:
data = pd.read_csv('/mnt/disk2/arshia.yousefinezhad/emotion_detection/data/preprocess_labelencoding_data.csv')

def preprocess_text(df):
    def process_text(text):
      text = re.sub(r'[^ا-ی!\s]', '', text)
      text = re.sub('[0-9]','',text)
      return text
    df.combined_text = df.combined_text.apply(process_text)
    return df

data_prepared = preprocess_text(data)

In [4]:
train, valid, test = split_data(data_prepared)

x_train, y_train = train['combined_text'].values.tolist(), train['emotion_id'].values.tolist()
x_valid, y_valid = valid['combined_text'].values.tolist(), valid['emotion_id'].values.tolist()
x_test, y_test = test['combined_text'].values.tolist(), test['emotion_id'].values.tolist()

(9306, 3)
(1035, 3)
(1149, 3)


## Config

In [5]:
parsbert_id = 'HooshvareLab/bert-fa-base-uncased'

label_list = list(sorted(data_prepared['emotion'].unique()))

label2id = {label: i for i, label in enumerate(label_list)}
id2label = {v: k for k, v in label2id.items()}

## Tokenizers

In [6]:
tokenizer = BertTokenizer.from_pretrained(parsbert_id)
bertcofig = BertConfig.from_pretrained(
    parsbert_id, **{
        'label2id': label2id,
        'id2label': id2label,
    })


model = TextClassificationParsBert(bertcofig, parsbert_id)
model = model.to(device)

In [7]:
train_data_loader = create_data_loader(train['combined_text'].to_numpy(), train['emotion'].to_numpy(), tokenizer, max_len, train_batch_size, label_list)
valid_data_loader = create_data_loader(valid['combined_text'].to_numpy(), valid['emotion'].to_numpy(), tokenizer, max_len, valid_batch_size, label_list)
test_data_loader = create_data_loader(test['combined_text'].to_numpy(), test['emotion'].to_numpy(), tokenizer, max_len, test_batch_size, label_list)

## ParsBert Model

In [8]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

torch_model = pytorch_model(model, criterion, optimizer, scheduler)
torch_model.trainer(train_data_loader, valid_data_loader, test_data_loader, epoch=1)

Epochs... : 100%|██████████| 1/1 [01:16<00:00, 76.62s/it]


-----------------------------------------------------------
| end of epoch   1 | time: 76.62s | valid accuracy    0.644 
-----------------------------------------------------------
Checking the results of test dataset.


100%|██████████| 9/9 [00:04<00:00,  2.02it/s]


test accuracy    0.644


In [11]:
text = "خیلی کیفت کوکه! چی شده؟,آره ، آزمون رانندگیمو قبول شدم \
اولین بار بود؟,نه بابا 5 بار رد شده بودم \
پس باید بهم شیرینی بدی,چشم، اینقد خوشحالم که میتونم یه شهرو شیرینی بدم \
حالا نمیخواد ولخرجی کنی ، شکم مارو سیر کنی کافیه,بزن  بریم تجریش یه ناهار مشتی بهت بدم \
بابا شرمنده میکنی,بیا بریم امروز فقط عشق و حاله"

model_torch_model = torch_model.model

model = model.to("cpu")

print("این هست یک حالت %s " % id2label[torch_model.predict(text, tokenizer)])

این هست یک حالت معمولی 
